# Steps
[ ] Run inferences on a different dataset and use activations to calculate one-hot concept vector for each image  
[ ] Store prediction as label  
[ ] Train decision tree  
[ ] Get metrics for accuracy of decision tree predictions and actual alexnet predictions  

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch import randperm
from torch._utils import _accumulate
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, sampler
from torch.utils.data.dataset import Subset
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm, tqdm_notebook
import pandas as pd

import graphviz



import skimage

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn import tree

%load_ext autoreload
%autoreload 2

In [2]:
def random_split(dataset, lengths):
    """
    Randomly split a dataset into non-overlapping new datasets of given lengths.

    Arguments:
        dataset (Dataset): Dataset to be split
        lengths (sequence): lengths of splits to be produced
    """
    if sum(lengths) != len(dataset):
        raise ValueError("Sum of input lengths does not equal the length of the input dataset!")

    indices = randperm(sum(lengths))
    return [Subset(dataset, indices[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]

In [3]:
data_dir = 'data/natural_images_small'

model_name = 'resnet50'

num_classes = 4

batch_size = 10

num_epochs = 5

feature_extract = True

In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
# initialize the model
model = models.resnet50(pretrained=True)

In [7]:
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.fc.in_features

# change classifier layer
model.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224 # how large the image should be

In [8]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# # Create training and validation datasets
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# # Create training and validation dataloaders


Initializing Datasets and Dataloaders...


In [9]:
all_data = datasets.ImageFolder(root=data_dir)
train_data_len = int(len(all_data)*0.8)
valid_data_len = int((len(all_data) - train_data_len))
train_data, val_data = random_split(all_data, [train_data_len, valid_data_len])
train_data.dataset.transform = data_transforms['train']
val_data.dataset.transform = data_transforms['val']
print(len(train_data), len(val_data))

2799 700


In [10]:
dataloaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=4), 
    'val': torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4) 
}

In [11]:
device = torch.device('cuda:0')
model = model.to(device)
print("model initialized and moved to GPU")

model initialized and moved to GPU


In [12]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [13]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()
dataloaders_dict = {}
# Train and evaluate
model, hist = train_model(
        model, dataloaders, criterion, optimizer_ft, 
        num_epochs=num_epochs, is_inception=(model_name=="inception")
    )

Epoch 0/4
----------
train Loss: 0.1742 Acc: 0.9589
val Loss: 0.0177 Acc: 0.9986

Epoch 1/4
----------
train Loss: 0.0570 Acc: 0.9879
val Loss: 0.0100 Acc: 0.9986

Epoch 2/4
----------
train Loss: 0.0423 Acc: 0.9921
val Loss: 0.0074 Acc: 0.9986

Epoch 3/4
----------
train Loss: 0.0345 Acc: 0.9925
val Loss: 0.0063 Acc: 0.9986

Epoch 4/4
----------
train Loss: 0.0294 Acc: 0.9939
val Loss: 0.0056 Acc: 0.9986

Training complete in 0m 55s
Best val Acc: 0.998571


In [14]:
# save trained model
torch.save(model.state_dict(), 'models/resnet50-natural-images-small.pt')

In [15]:
# max pool, flattens, then normalizes activations
def transform_activations(activations):
    m = nn.AvgPool2d(4)
    max_pool_outputs = m(activations)
    flattened_outputs = max_pool_outputs.reshape((max_pool_outputs.shape[0], np.prod(max_pool_outputs.shape[1:])))
    normalized_outputs = normalize(flattened_outputs)
    return normalized_outputs
    

In [16]:
activations = []
classes = []
# define layer hook
def layer_hook(module, input, output):
    layer_output = output.data.cpu()
    activations.append(layer_output)

In [17]:
layer = 'layer3'
# register hook on layer
# handle = model.layer1[2].bn3.register_forward_hook(layer_hook)
# handle = model.layer2[3].bn3.register_forward_hook(layer_hook)
handle = model.layer3[5].bn3.register_forward_hook(layer_hook)
# handle = model.layer4[2].bn3.register_forward_hook(layer_hook)

In [18]:
activations = []
classes = []

activation_dict = {}

# setup for evaluation mode
model.eval()

# do same thing for training set
for inputs, labels in dataloaders['train']:
    inputs = inputs.to(device)
    outputs = model(inputs).data.cpu()
    _, preds = torch.max(outputs, 1)

    classes.append(preds)

activations = torch.cat(activations, dim=0)
classes = torch.cat(classes, dim=0)

activation_dict['train'] = (transform_activations(activations), classes)


# reset and do the same for val set
activations = []
classes = []

# do same thing for val set
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    outputs = model(inputs).data.cpu()
    _, preds = torch.max(outputs, 1)

    classes.append(preds)

activations = torch.cat(activations, dim=0)
classes = torch.cat(classes, dim=0)

activation_dict['val'] = (transform_activations(activations), classes)




In [19]:
handle.remove()

In [20]:
activations = activation_dict

In [21]:
model.eval()
print("model in evaluation mode...")

model in evaluation mode...


In [22]:
activations['train'][0].shape

(2799, 9216)

In [23]:
import pickle
with open('pca/%s.pickle' % layer, 'rb') as handle:
    pca = pickle.load(handle)

In [24]:
train_acts = pca.transform(activations['train'][0])
val_acts = pca.transform(activations['val'][0])
Y_train = activations['train'][1]
Y_val = activations['val'][1]

In [25]:
import pickle

# Load data (deserialize)
with open('classifiers-%s.pickle' % layer, 'rb') as handle:
    classifiers = pickle.load(handle)

In [26]:
concepts_to_remove = [93, 19, 38, 79]
for c in concepts_to_remove:
    if classifiers.get(c) is not None:
        del classifiers[c]

In [27]:
concepts = list(classifiers.keys())

In [28]:
len(concepts)

97

In [35]:
def get_concept_vectors(activations, classifiers, concepts=[]):
    predictions = []
    
    # if you specify which concepts to measure:
    if len(concepts) > 0:
        for concept in concepts:
            clf = classifiers.get(concept)[0]
            pred = clf.predict(activations)[np.newaxis]
            pred = np.transpose(pred)
            predictions.append(pred)

    else:
        num_concepts = len(classifiers.keys())

        # for each classifier, get the prediction
        for clf in classifiers.items():
            pred = clf[1][0].predict(activations)[np.newaxis]
            pred = np.transpose(pred)
            predictions.append(pred)
    
    return np.hstack(predictions)

In [30]:
# filter_concepts = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,44,48,52,57,60]
# filter_concepts = list(set(filter_concepts) & set(concepts))

In [36]:
X_train = get_concept_vectors(train_acts, classifiers)
X_val = get_concept_vectors(val_acts, classifiers)


In [37]:
labels_frame = pd.read_csv('dataset/broden1_227/label.csv')
concept_names = [labels_frame.at[x-1, 'name'] for x in concepts]
class_names = train_data.dataset.classes

In [38]:
class_names

['car', 'dog', 'flower', 'person']

In [39]:
len(concepts)

97

In [35]:
# for n in range(20):#len(X_train):
#     names = [concept_names[i] for i, x in enumerate(X_train[n]) if x > 0]
#     print(len(names),names,class_names[Y_train[n]])

In [40]:
def train_tree(name, X_train, X_val, Y_train, Y_val, min_samples, max_depth, concept_names, class_names, export=False, sample_weights=None):
    decision_tree = tree.DecisionTreeClassifier(min_samples_leaf=min_samples, max_depth=max_depth, criterion='gini', min_impurity_split=0.1)
    decision_tree = decision_tree.fit(X_train, Y_train, sample_weight=sample_weights)
    score = decision_tree.score(X_val, Y_val)
    print("%f" % score)
    dot_data = tree.export_graphviz(
        decision_tree, out_file=None, feature_names=concept_names, class_names=train_data.dataset.classes, 
        filled=True, leaves_parallel=False, impurity=False, rounded=True, label='none'
    )
    graph = graphviz.Source(dot_data)
    
    important_features = decision_tree.feature_importances_
    
    if export:
        file_name = "%s-%i-%i-%s" % (name, max_depth, min_samples, ("%.4f" % round(score,4)).split('.')[1])
        print("outputting file to %s" % file_name)
        graph.render("trees/natural_images_small/%s/%s"% (layer, file_name))

    

In [41]:
def generate_sample_weights(data):
    weights = []
    scores = [value[1] for key, value in classifiers.items()]
    for d in data:
        avg_sum = 0
        count = 0
        avg = 0
        for i, concept in enumerate(d):

            if (concept == 1):
                avg_sum += scores[i]
                count+=1
        if(count != 0):            
            avg = avg_sum / count
            
        if(count == 0):
            weights.append(0)
        else:
            weights.append(avg)
    
    return weights
            

In [42]:
sample_weights=generate_sample_weights(X_train)

In [46]:
MIN_SAMPLES = 20
MAX_DEPTH = 5
EXPORT = True
# train_tree('natural_images_small_layer3_weighted', X_train, X_val, Y_train, Y_val, MIN_SAMPLES, MAX_DEPTH, concept_names, class_names, export=EXPORT, sample_weights=sample_weights)
train_tree('natural_images_small', X_train, X_val, Y_train, Y_val, MIN_SAMPLES, MAX_DEPTH, concept_names, class_names, export=EXPORT)

0.912857
outputting file to natural_images_small-5-20-9129


/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


In [40]:
min_sample_vals = [5, 20]
max_depth_vals = [3, 4, 5, 10, 20, 50]

for min_sample in min_sample_vals:
    for max_depth in max_depth_vals:
        train_tree('natural_images_small', X_train, X_val, Y_train, Y_val, min_sample, max_depth, concept_names, class_names, export=EXPORT)

/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/home/conner/anaconda3/envs/torch2

0.894286
0.908571
0.918571
0.924286
0.927143
0.927143
0.894286
0.908571
0.900000
0.900000
0.900000
0.900000


/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/home/conner/anaconda3/envs/torch2/lib/python2.7/site-packages/sklearn/tree/tree.py:283: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


In [41]:
len(concepts)

97

In [43]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [ ]:
accuracy_df = pd.DataFrame(data=([concept_names[c] for c in concepts[:25]], [value[1] for key, value in classifiers.items()][:25]))

In [188]:
plt.bar([concept_names[c] for c in concepts[:20]], [value[1] for key, value in classifiers.items()][:20], width=0.5, align='center')
plt.title('Acurracies of Top 20 Concepts for Layer 3 of Resnet50')
plt.xlabel('Concepts')
plt.ylabel('Accuracy')
plt.xticks(rotation=80)
plt.savefig('test.pdf', bbox_inches='tight')

In [182]:
classifiers.get(29)

(SGDClassifier(alpha=0.0001, average=True, class_weight=None,
        early_stopping=True, epsilon=0.1, eta0=1.5, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='hinge',
        max_iter=150.0, n_iter=None, n_iter_no_change=5, n_jobs=None,
        penalty='l2', power_t=0.5, random_state=None, shuffle=True,
        tol=None, validation_fraction=0.1, verbose=0, warm_start=False),
 0.8942307692307693)